In [ ]:
%matplotlib inline
import os
import matplotlib.pyplot as plt
os.environ['THEANO_FLAGS'] = 'device=gpu'
import numpy as np
from keras.layers import *
from keras.layers import Dense, Input, Dropout
from keras.models import Sequential, Model
from keras.optimizers import Adadelta, RMSprop
from sklearn.datasets import make_blobs
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from keras.regularizers import l2, l1

In [ ]:
dataset = np.load('data.npz')
X_full = dataset['X']
X_full = X_full.reshape((X_full.shape[0], -1))
y = dataset['nMCparticles']
y_orig = y.copy()
#y = y > 6
y = to_categorical(y) 

X_full = X_full.reshape((X_full.shape[0], 18, 18, 30))
#X_full = X_full.transpose((0, 3, 1, 2))



indices = np.arange(len(X_full))
np.random.shuffle(indices)
X_full = X_full[indices]
y_full = y[indices]
#X_full[X_full>0] = 1
#X_full[X_full==0] = -1



In [ ]:
plt.hist(y_orig)

In [ ]:
X_train = X_full[0:3000]
y_train = y_full[0:3000]
X_valid = X_full[3000:]
y_valid = y_full[3000:]

In [ ]:
def build_model():
    
    inp = Input(shape=(X_full.shape[1], X_full.shape[2], X_full.shape[3]))
    
    x = inp
    
    
    x = Convolution2D(16, 3, 3, activation='relu')(x)
    x = Convolution2D(16, 3, 3, activation='relu')(x)
    x = Convolution2D(16, 3, 3, activation='relu')(x)
    x = Convolution2D(32, 3, 3, activation='relu')(x)
    x = Convolution2D(32, 3, 3, activation='relu')(x)
    x = Convolution2D(32, 3, 3, activation='relu')(x)
    x = Convolution2D(64, 3, 3, activation='relu')(x)
    x = Convolution2D(64, 3, 3, activation='relu')(x)
    
    x = Flatten()(x)
    
    #x = Dense(100, activation='relu')(x)
    #x = Dense(100, activation='relu')(x)
    out = Dense(y.shape[1], activation='softmax')(x)

    model = Model(input=inp, output=out)
    optimizer = RMSprop(lr=0.0001)
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
model = build_model()

In [ ]:
for i in range(30):
    X_train_ = X_train.copy()

    """

    ind_z = np.random.choice((0, 1), size=X_train.shape[0])
    X_train_[ind_z] = X_train_[ind_z, ::-1, :, :]

    ind_y = np.random.choice((0, 1), size=X_train.shape[0])
    X_train_[ind_y] = X_train_[ind_y, :, ::-1, :]
    
    
    ind_x = np.random.choice((0, 1), size=X_train.shape[0])
    X_train_[ind_x] = X_train_[ind_x, :, :, ::-1]
    """

    model.fit(X_train_, y_train, 
              verbose=1, 
              validation_data=(X_valid, y_valid),
              callbacks=[EarlyStopping(monitor='val_acc', patience=10, verbose=1, mode='max')],
              nb_epoch=1)

In [ ]:
w = model.layers[2].W.get_value()
print(w.shape)
w = w[:, 7]
w = w.reshape((18, 18, 30))
for i in range(30):
    fig = plt.figure(figsize=(2, 2))
    plt.imshow(w[:, :, i], cmap='gray')
    plt.show()